In [1]:
%matplotlib inline

In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.3.0


In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: trialws3
Azure region: eastus
Subscription id: cb4187b1-dbe9-459d-a2f7-dfa57764a96f
Resource group: trialws3


In [4]:
from azureml.core.dataset import Dataset

dataset_paths = [
    'https://raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_idle.txt',
    'https://raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_idle_2.txt',
    'https://raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_working.txt',
    'https://raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_working_2.txt'
]

dataset = Dataset.File.from_files(path = dataset_paths)

In [5]:
dataset_registered = False
try:
    temp = Dataset.get_by_name(workspace=ws, name='motor_activity_recognition')
    dataset_registered = True
except:
    print("The dataset motor activity is not registered in workspace yet.")

if not dataset_registered:
    dataset = dataset.register(workspace = ws,
                               name = 'motor_activity_recognition',
                               description='training and test dataset for motor activity recognition',
                               create_new_version=True)

In [6]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

c1ds2v2 ComputeInstance Succeeded


In [7]:
dataset = Dataset.get_by_name(ws, 'motor_activity_recognition')

# list the files referenced by mnist dataset
dataset.to_path()

['/https/raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_idle.txt',
 '/https/raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_idle_2.txt',
 '/https/raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_working.txt',
 '/https/raw.githubusercontent.com/alwint3r/Motor-Activity-Detection-TinyML/master/output_label_working_2.txt']

In [8]:
data_folder = dataset.as_named_input('motor_activity').as_mount()

In [ ]:
data_folder.dataset

In [10]:
from azureml.core import Experiment

script_folder = './motor-activity-recognition'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='motor-activity-recognition-2')

In [11]:
import shutil

# the training logic is in the keras_mnist.py file.
shutil.copy('./tfkeras-train-model.py', script_folder)

'./motor-activity-recognition/tfkeras-train-model.py'

In [12]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.environment import CondaDependencies
from azureml.core import Environment

env = Environment(name="motor_activity_env")
env.docker.enabled = True

conda_dep = CondaDependencies()
conda_dep.add_pip_package("scipy")
conda_dep.add_pip_package("scikit-learn")
conda_dep.add_pip_package("pandas")
conda_dep.add_pip_package("numpy==1.16.2")
conda_dep.add_pip_package("matplotlib==3.0.3")
conda_dep.add_pip_package("tensorflow==2.1.0")

env.python.conda_dependencies = conda_dep


In [19]:
from azureml.core import ScriptRunConfig


script_params = [
    '--data-folder', dataset.as_named_input('motor_activity').as_mount(),
    '--batch-size', 50,
    '--n-fc-layer', 128,
    '--dropout-ratio', 0.1,
    '--learning-rate', 0.1
]

src = ScriptRunConfig(source_directory=script_folder,
                      script='tfkeras-train-model.py',
                      arguments=script_params)
src.run_config.environment = env
src.run_config.target = compute_targets

In [20]:
run = exp.submit(src)

In [21]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [23]:
best_run_id = None
best_run_accuracy = None

for run in exp.get_runs():
    metrics = run.get_metrics()
    details = run.get_details()
    
    run_id = details["runId"]
    
    try:
        accuracy = metrics["Final test accuracy"]
    except:
        pass
    
    if best_run_accuracy is None:
        best_run_accuracy = accuracy
        best_run_id = run_id
    else:
        if accuracy > best_run_accuracy:
            best_run_accuracy = accuracy
            best_run_id = run_id

print('Best run ID: {}'.format(best_run_id))
print('Best run accuracy: {}'.format(best_run_accuracy))

Best run ID: motor-activity-recognition-2_1587804446_be2a7838
Best run accuracy: 1.0


In [24]:
from azureml.core import Run
best_run = Run(experiment=exp, run_id=best_run_id)
print(best_run.get_file_names())

['Accuracy vs Loss_1587804484.png', 'azureml-logs/60_control_log.txt', 'azureml-logs/70_driver_log.txt', 'logs/azureml/9_azureml.log', 'outputs/model/model.h5', 'outputs/model/model.json', 'outputs/model/model.tflite']


In [25]:
best_run.download_file(name="outputs/model/model.tflite")